In [1]:
import numpy as np
import pandas as pd
import random

#aktivasyon fonksiyonu
def sigmoid(x):

    return 1/(1 + np.exp(-x))

#türevi
def der_sigmoid(x):
    return np.exp(x)/((np.exp(x)+1)**2)

#datayı okuyalım
data = pd.read_csv("hungary_chickenpox.csv", sep=';')
data["BUDAPEST"] = data["BUDAPEST"] * ((0.9-0.1) / (max(data["BUDAPEST"]) - min(data["BUDAPEST"]))) + 0.1

data = data.to_numpy()

#test ve eğitim kümelerine ayıralım
train_data = data[:400]
test_data = data[400:]

#gizli katman sayısı ve nöron sayılarını belirleyelim
input_dimension = 6
gk_1 = 20
gk_2 = 35
out = 1

#başlangıç ağırlıkları
weights_1 = (np.random.rand(gk_1, input_dimension + 1)-0.5)/1
weights_2 = (np.random.rand(gk_2, gk_1 + 1)-0.5)/1
weights_out = (np.random.rand(out, gk_2 + 1)-0.5)/1

#momentum terimi için kullanılacak bir önceki adım ağırlıkları
weights_1_old = 0
weights_2_old = 0
weights_out_old = 0

Error = 0
n_learn = 0.01
alpha = 0.9
lower_noise = 0.01
#noise'ı verilerimize oranla küçültünce hata baya düştü
upper_noise = 0.04

#5 terim geriye gidilerek yapılan eğitim
for j in range(500):
    Error = 0
    for i in range(6, len(train_data)):
        step_data = [train_data[i-5][1],
                     train_data[i-4][1],
                     train_data[i-3][1],
                     train_data[i-2][1],
                     train_data[i-1][1],
                     round(random.uniform(lower_noise, upper_noise), 2)]

        step_data = np.append(step_data, [1])
        vgk1_step = np.matmul(weights_1, step_data)
        y1 = sigmoid(vgk1_step)
        y1 = np.append(y1, 1)

        vgk2_step = np.matmul(weights_2, y1)
        y2 = sigmoid(vgk2_step)
        y2 = np.append(y2, 1)

        vo_step = np.matmul(weights_out, y2)
        yo = sigmoid(vo_step)
        
        step_error = yo - train_data[i][1]
        
        #yerel gradyenler
        grd_o = step_error * der_sigmoid(vo_step)
        grd_2 = np.matmul(np.transpose(weights_out)[:-1], grd_o) * der_sigmoid(vgk2_step)
        grd_1 = np.matmul(np.transpose(weights_2)[:-1], grd_2) * der_sigmoid(vgk1_step)
        #ağırlık güncelleme
        weights_1_new = weights_1 - n_learn * np.matmul(grd_1.reshape(-1, 1), np.transpose(step_data.reshape(-1, 1))) + alpha * (weights_1 - weights_1_old)
        weights_2_new = weights_2 - n_learn * np.matmul(grd_2.reshape(-1, 1), np.transpose(y1.reshape(-1, 1))) + alpha * (weights_2 - weights_2_old)
        weights_out_new = weights_out - n_learn * np.matmul(grd_o.reshape(-1, 1), np.transpose(y2.reshape(-1, 1))) + alpha * (weights_out - weights_out_old)
        #bir önceki ağırlıkları tutma
        weights_1_old = weights_1
        weights_2_old = weights_2
        weights_out_old = weights_out
        #güncellenmiş ağırlıkları kaydedilmesi
        weights_1 = weights_1_new
        weights_2 = weights_2_new
        weights_out = weights_out_new

        Error = Error + 0.5*np.matmul(step_error, np.transpose(step_error))

    if Error/len(train_data) < 0.001:   #durdurma kriteri
        print(j)
        break

print(Error/len(train_data))

#test aşaması
Error = 0
for i in range(6,len(test_data)):
    step_data = step_data = [test_data[i-5][1],
                     test_data[i-4][1],
                     test_data[i-3][1],
                     test_data[i-2][1],
                     test_data[i-1][1],
                     round(random.uniform(lower_noise, upper_noise), 2)]
    step_data = np.append(step_data, [1])

    vgk1_step = np.matmul(weights_1, step_data)
    y1 = sigmoid(vgk1_step)
    y1 = np.append(y1, 1)

    vgk2_step = np.matmul(weights_2, y1)
    y2 = sigmoid(vgk2_step)
    y2 = np.append(y2, 1)

    vo_step = np.matmul(weights_out, y2)
    yo = sigmoid(vo_step)

    step_error = yo - test_data[i][1]

    Error = Error + 0.5*np.matmul(step_error, np.transpose(step_error))

print(Error/len(test_data))

0.0033509898245660794
0.004732209859001704


In [2]:
1-Error/len(test_data)

0.9952677901409983